In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
@everywhere using InvertedIndices
@everywhere using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

In [3]:
gmat = total_symm_calc(positions,dis2mat,f_mat,total_symm_vec)
@benchmark total_symm_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 110 samples with 1 evaluation.
 Range (min … max):  40.946 ms … 53.003 ms  ┊ GC (min … max):  9.18% … 7.34%
 Time  (median):     44.150 ms              ┊ GC (median):     8.90%
 Time  (mean ± σ):   45.494 ms ±  2.961 ms  ┊ GC (mean ± σ):  10.90% ± 3.15%

            ▂█▂▄                                               
  ▃▁▃▁▁▃▅▃███████▆▃▆▆▃▇▁▃▃▃▁▅▅▇▁▁▁▅▃▃▅▃▃▃▅▁▇▅▁▃▅▁▃▅▅▃▃▅▁▁▁▃▁▃ ▃
  40.9 ms         Histogram: frequency by time        52.6 ms <

 Memory estimate: 104.19 MiB, allocs estimate: 1364455.

In [4]:
file = open("$(pwd())/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])
truevals ≈ gmat

# X=gmat.-truevals

true

# Testing the waters with iterating over symmetry functions

NB This method failed. It was significantly slower due to the iterators and memory access.

In [23]:
perturbation = [-.1 ,.3,.2 ]
newpos = positions[3] .+ perturbation
index = 3

new_dis_vec = [distance2(newpos,b) for b in positions]
new_dis_vec[3] = 0.

new_f_vec = cutoff_function.(sqrt.(new_dis_vec),Ref(total_symm_vec[1].r_cut))


55-element Vector{Float64}:
 0.6729622810620872
 0.6673537611763615
 1.0
 0.641313952432365
 0.25867027813036586
 0.25848314873322153
 0.6407783065380429
 0.6000753639415433
 0.23563090780103746
 0.1043164272069067
 ⋮
 0.0
 0.08540286302967126
 0.15905660612244632
 0.0780399503184917
 0.0019532999653086303
 0.0
 0.0019323260080338356
 0.07794636305290437
 0.0

In [24]:
gmat = total_symm_calc(positions,newpos,dis2mat,new_dis_vec,f_mat,new_f_vec,gmat,3,total_symm_vec)

88×55 Matrix{Float64}:
 -2200.69       212.214     7147.31     …  292.602        260.957
     0.0          0.0          0.0           0.0            0.0
 -1481.01       249.533     1454.45        104.762         52.8125
     0.0          0.0          0.0           0.0            0.0
 -1082.52       244.036      377.893        44.0032        15.6562
     0.0          0.0          0.0      …    0.0            0.0
  -277.729      130.884      224.959         1.17372        0.462298
     0.0          0.0          0.0           0.0            0.0
    -0.519322     0.860443     3.84174       0.000528538    0.000557398
     0.0          0.0          0.0           0.0            0.0
     ⋮                                  ⋱                 
   NaN            2.1881     NaN             0.00374904     0.00170206
     0.0          0.0          0.0      …    0.0            0.0
     0.0          0.0          0.0           0.0            0.0
   NaN            1.62474    NaN             0.000451767  

In [7]:
@benchmark total_symm_calc($positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$gmat,$3,$total_symm_vec)

BenchmarkTools.Trial: 701 samples with 1 evaluation.
 Range (min … max):  1.622 ms … 88.056 ms  ┊ GC (min … max):  0.00% … 96.05%
 Time  (median):     3.508 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.122 ms ± 13.608 ms  ┊ GC (mean ± σ):  43.76% ± 20.81%

  ██▆▅▃▂                                                      
  ███████▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▁▆▅▇▆▄▆▁▄▄▄▄▄▄ ▇
  1.62 ms      Histogram: log(frequency) by time     72.8 ms <

 Memory estimate: 28.46 MiB, allocs estimate: 372425.

In [8]:
function testspeed(positions,new_pos,dist2_mat,new_dist_vec,f_mat,new_f_vec,g_mat,atomindex,total_symm_vec)
    for i in 1:100
        total_symm_calc(positions,new_pos,dist2_mat,new_dist_vec,f_mat,new_f_vec,g_mat,atomindex,total_symm_vec)
    end
end
@benchmark testspeed($positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$gmat,$3,$total_symm_vec)

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  644.279 ms … 796.471 ms  ┊ GC (min … max): 45.28% … 43.99%
 Time  (median):     717.085 ms               ┊ GC (median):    41.92%
 Time  (mean ± σ):   720.275 ms ±  56.369 ms  ┊ GC (mean ± σ):  42.81% ±  1.46%

  █      █       █      █             █           █    █      █  
  █▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁█ ▁
  644 ms           Histogram: frequency by time          796 ms <

 Memory estimate: 2.78 GiB, allocs estimate: 37242689.

In [9]:
@benchmark all_angular_measure($positions[1],$positions[2],$positions[5],$dis2mat[1,2],$dis2mat[1,5],dis2mat[2,5])

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  42.155 ns …   6.103 μs  ┊ GC (min … max):  0.00% … 98.99%
 Time  (median):     46.881 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   63.304 ns ± 274.765 ns  ┊ GC (mean ± σ):  21.20% ±  4.84%

  ▁▅▆▇███▇▆▅▄▄▃▃▂▂▁▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁     ▁▁ ▁▁                   ▃
  ███████████████████████████████████▇████████▇▆▇▇▇▇▆▇▇▅▄▅▃▅▂▄ █
  42.2 ns       Histogram: log(frequency) by time      87.8 ns <

 Memory estimate: 224 bytes, allocs estimate: 7.

# A new approach

Given the obvious limitations of the current approach and the issues previously faced, we acknowledge that three times the original vectors are generated at each step, this causes major problems with memory allocation and garbage collection, resulting in slower calculation of symmetry funcitons overall. 

The aim of the next section is therefore to rewrite the code in a way that modifies existing arrays in place in an effort to reduce allocation overheads. 

In [10]:
exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk) = exp(-η*(r2_ij+r2_ik+r2_jk))* f_ij * f_ik * f_jk
theta_part(θ,λ,ζ) = (1+λ*θ)^ζ

theta_part (generic function with 1 method)

In [19]:
angsymmvec = []
for symmfunc in angularsymmvec
    values = [symmfunc.eta,symmfunc.lambda,symmfunc.zeta,symmfunc.tpz]
    push!(angsymmvec,values)
end
angsymmvec

78-element Vector{Any}:
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, 1.0, 1.0, 1.0]
 [0.0001, -1.0, 2.0, 0.5]
 [0.0001, -1.0, 2.0, 0.5]
 [0.0001, -1.0, 2.0, 0.5]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 1.0, 1.0]
 [0.003, -1.0, 2.0, 0.5]
 ⋮
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 2.0, 0.5]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, -1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]
 [0.08, 1.0, 4.0, 0.125]

In [20]:
function adjust_symm_val!(g_value,θ_val,exp_part,tpz)
    g_value +=exp_part*θ_val*tpz
    return g_value
end
function adjust_symm_val!(g_value,exp_part,θ,η,λ,ζ,tpz)
    θ_val = theta_part(θ,λ,ζ)

    return adjust_symm_val!(g_value,θ_val,exp_part,tpz)
end

function calc_symmetry_val(g_vector,θ_vec,indices,exp_part,η,λ,ζ,tpz)

    for atomicindex in eachindex(θ_vec)

        g_vector[indices[atomicindex]] = adjust_symm_val!(g_vector[indices[atomicindex]],exp_part,θ_vec[atomicindex],η,λ,ζ,tpz)

    end

    return g_vector
end
function calc_symmetry_val(g_vector,positions,dist2_mat,f_mat,indices,η,λ,ζ,tpz)
    
    θ_vec = 
    exp_part=

    return calc_symmetry_val(g_vector,θ_vec,indices,exp_part,η,λ,ζ,tpz)
end

calc_symmetry_value (generic function with 1 method)